In [2]:
def connect_db():

    engine = create_engine(connection_string, pool_recycle = 3600, encoding='utf-8')
    Session = sessionmaker(bind=engine, expire_on_commit= True)

    session = Session()
    return session

In [37]:
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, CHAR, Date, String, Time, Index, DateTime, TIMESTAMP, func
from sqlalchemy.dialects.mysql import INTEGER, BIT, VARCHAR, DATE, TINYINT, TIME, DOUBLE, TEXT
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy import PrimaryKeyConstraint
from sqlalchemy import ForeignKeyConstraint
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class Movie_Inf(Base):
    __tablename__ = 'movie_inf'

    movie_id        = Column(Integer, primary_key = True)
    m_name_k        = Column(VARCHAR(50))
    grade           = Column(VARCHAR(20))
    is_original     = Column(Integer)
    viewing_time    = Column(Integer)
    premier         = Column(DATE)
    nation          = Column(VARCHAR(10))
    m_name_e        = Column(VARCHAR(50))
    project_budget  = Column(Integer)
    gross_kor       = Column(Integer)
    gross_us        = Column(Integer)
    
    def set_movie_id(self,movie_id):
        self.movie_id = movie_id
    def set_m_name_k(self,m_name_k):
        self.m_name_k = m_name_k
    def set_grade(self,grade):
        self.grade = grade
    def set_is_original(self,is_original):
        self.is_original = is_original    
    def set_viewing_time(self,viewing_time):
        self.viewing_time = viewing_time 
    def set_premier(self,premier):
        self.premier = premier 
    def set_nation(self,nation):
        self.nation= nation
    def set_m_name_e(self,m_name_e):
        self.m_name_e= m_name_e 
    def set_project_budget(self,project_budget):
        self.project_budget = project_budget
    def set_gross_kor(self,gross_kor):
        self.gross_kor = gross_kor
    def set_gross_us(self,gross_us):
        self.gross_us = gross_us  
    
class Genre(Base):
    __tablename__ = 'genre'
    
    movie_id        = Column(Integer, ForeignKey("movie_inf.movie_id"))
    genre           = Column(VARCHAR(20), primary_key = True)
    
    def __init__(self,movie_id,genre):
        self.movie_id = movie_id
        self.genre = genre
    def set_movie_id(self,movie_id):
        self.movie_id = movie_id
    def set_genre(self,genre):
        self.genre = genre


class Director(Base):
    __tablename__ = 'director'
    
    director_id     = Column(Integer, primary_key = True)
    movie_id        = Column(Integer, ForeignKey("movie_inf.movie_id"))
    director        = Column(VARCHAR(30))
    
    def set_director_id(self,director_id):
        self.director_id = director_id
    def set_movie_id(self,movie_id):
        self.movie_id = movie_id
    def set_director(self,director):
        self.director = director
    
class Director_Movies(Base):
    __tablename__ = 'director_movies'
    
    director_id     = Column(Integer, ForeignKey("director.director_id"))
    director_movie  = Column(VARCHAR(50), primary_key = True)
    gross           = Column(Integer)
    premier         = Column(DATE)
    
    def set_director_id(self,director_id):
        self.director_id = director_id
    def set_director_movie(self,director_movie):
        self.director_movie = director_movie
    def set_gross(self,gross):
        self.gross = gross
    def set_premier(self,premier):
        self.premier = premier
        
class Actor(Base):
    __tablename__ = 'actor'
    
    actor_id        = Column(Integer, primary_key = True)
    movie_id        = Column(Integer, ForeignKey("movie_inf.movie_id"))
    actor           = Column(VARCHAR(30))
    
    def set_actor_id(self,actor_id):
        self.actor_id = actor_id
    def set_movie_id(self,movie_id):
        self.movie_id = movie_id
    def set_actor(self,actor):
        self.actor = actor
    
class Actor_Movies(Base):
    __tablename__ = 'actor_movies'
    
    actor_id     = Column(Integer, ForeignKey("actor.actor_id"))
    actor_movie  = Column(VARCHAR(80), primary_key = True)
    gross        = Column(Integer)    
    premier      = Column(DATE)
    
    def set_actor_id(self,actor_id):
        self.actor_id = actor_id
    def set_actor_movie(self,actor_movie):
        self.actor_movie = actor_movie
    def set_gross(self,gross):
        self.gross = gross
    def set_premier(self,premier):
        self.premier = premier
    
class Franchise(Base):
    __tablename__ = 'franchise'
    
    movie_id     = Column(Integer, ForeignKey("movie_inf.movie_id"))
    franchise_movie   = Column(VARCHAR(50), primary_key = True)
    gross        = Column(Integer)    
    
    def set_movie_id(self,movie_id):
        self.movie_id = movie_id
    def set_franchise_movie(self,franchise_movie):
        self.franchise_movie = franchise_movie
    def set_gross(self,gross):
        self.gross = gross

In [1]:
import MySQLdb
import json
import re

def load_movie_names(file_name): 
    
    f = open(file_name, "r")
    movie_data = json.load(f)
    f.close()
    
    return movie_data


In [4]:
def insert_into_db(class_name):
    
    session = connect_db()
    session.add(class_name)
    session.commit()
    
def update_into_db(class_name, filter_condition, update_dict):
    session = connect_db()
    
    rows = session.query(class_name).filter(filter_condition).update(update_dict)
    session.commit()

def select_from_db(class_name, filter_condition):
    
    session = connect_db()
    result = session.query(class_name).filter(filter_condition).all()

    
    return result

# Naver API 이용해서 영화 이름으로(영어) query 보냄

In [3]:
import requests
import json
import re

def request_naver_movie(movie_name):
    
#    client_key = 
#    client_secret =

    headers = {}
    headers['Content-Type'] = 'application/xml'
    headers['X-Naver-Client-Id'] = client_key
    headers['X-Naver-Client-Secret'] = client_secret
    
    url = 'https://openapi.naver.com/v1/search/movie.json?query={}'.format(movie_name)
    
    try:

        res = requests.get(url)

        res = requests.get(url, headers = headers)
        data = json.loads(res.text)
    
        #검색결과 없으면 : 기호 떼고 다시 검색
        if not data['items']:
            
            temp = re.split(r'[:]',movie_name)
            modify_movie_name = temp[0]

            data = request_naver_movie(modify_movie_name)
    except:
        data = None
    return data

# query 통해서 가장 가능성 높은 naver link 가져옴

In [6]:

def get_naver_links(movie_name):
    
    data = request_naver_movie(movie_name)
    
    #링크
    naver_movie_link = data['items'][0]['link']
        
    return naver_movie_link


def get_korean_title(movie_name):

    data = request_naver_movie(movie_name)

    #영화 제목(한글)
    m_name_k = data['items'][0]['title']    
    m_name_k = re.sub(r'(<b>)|(</b>)','',m_name_k)   #<b> 기호 떼기 
    
    return m_name_k

In [7]:
import requests
from bs4 import BeautifulSoup
import sys

def crawling_naver_movie(url):

    res = requests.get(url)
    soup = BeautifulSoup(res.content)

    body = soup.find('dl', attrs = {'class' : 'info_spec'})

    dd = body.select('dd')[0]
    span = dd.select('span')
    
    return span

def crawling_mojo_movie(sub_url):
    
    url = 'http://www.boxofficemojo.com{}'.format(sub_url)
    
    res = requests.get(url)
    soup = BeautifulSoup(res.content)
    
    return soup

# 장르, 상영시간, 개봉일, 국가 정보 크롤링

In [8]:
def get_naver_genre(span):

    try:
    
        temp = re.sub(r'[\s]','',span[0].get_text())  #공백제거
        temp = re.split(r'[,]',temp)  #다중 값 분리
        #장르 저장
        genre_list=[]
        for i in range(len(temp)):
            g = temp[i]
            genre_list.append(g)
    except IndexError:
        genre_list= None
    return genre_list

def get_naver_viewing_time(span):
    try:
        #상영시간 저장
        temp = re.split('분', span[2].get_text().encode('utf-8'))
        viewing_time = int(temp[0])
       
    except:
        viewing_time =None
        
    return viewing_time

def get_naver_premier(span):
    try:
        #개봉일 저장
        
        temp = re.sub(r'개봉','',span[3].get_text().encode('utf-8'))
        temp = re.sub(r'[\s]','',temp)
        temp = re.sub(r'[.]','-',temp)
        premier = temp

    except:
        premier =None
    return premier

def get_naver_nation(span):
    try:
        #국가 저장
        
        temp = re.sub(r'[\s]','',span[1].get_text())  
        temp = re.split(r'[,]',temp) 
        
        nation = temp[0]

    except IndexError:
        nation = None
    return nation

In [95]:
def get_movie_grade(soup): # saved_movie_dict는 이전에 저장한 dict형식의 영화정보

    movie_grades = soup.select('html > body > div#wrap > div#container > div#content > div.article > div.mv_info_area > div.mv_info > dl.info_spec > dd > p > a[href*="grade"]')
    if len(movie_grades) >= 1:
        grade = movie_grades[0].get_text()
    else :
        grade = None
    return grade

def get_movie_origin(soup): #원작 존재하면 1, 없으면 0
    movie_origin = soup.select('html > body > div#wrap > div#container > div#content > div.article > div.mv_info_area > div.mv_info > div.btn_sns > div.end_btn_area > ul > li > a[href*="bookdb"]')
    if len(movie_origin) >= 1:
        is_original = 1
    else :
        is_original = 0
    return is_original

def get_k_movie_title(soup): # 영화제목 추출
    h3 = soup.find('h3', attrs = {'class':'h_movie'})
    a = h3.select('a')
    m_name_k = a[0].get_text()
   
    return m_name_k

In [32]:
import datetime as dt

def get_project_budget(soup):
    try:

        body = soup.select('tbody')[5]
        b = body.select('b')[6]
        project_budget = re.sub(r'[$\smillion]','',b.get_text())
        project_budget = int(project_budget)* 1000000
        
    except:
        project_budget = None
    return project_budget

def get_gross_kor(soup):
    reg = r'^/movies/\?page=intl&country=KR&id=.+.htm'
    try:
        all_a = soup.find_all('a', href= True)
        for a in all_a:
            if re.findall(reg,a['href']):
                tag_a = a

        string = tag_a.find_parent('tr').get_text()
        temp =re.split(r'\n',string)
        if re.findall(r'[$]',temp[5]):
            gross_kor = re.sub(r'[$,]','',temp[5])            
        else:
            gross_kor = re.sub(r'[$,]','',temp[4])
            
        gross_kor = int(gross_kor)

    except:
            gross_kor = None
    return gross_kor

def get_gross_us(soup):
    try:
    
        div = soup.find('div', attrs = {'class' : 'mp_box_content'})
        b = div.select('b')[1]
        gross_us = re.sub(r'[$,]','',b.get_text())
        gross_us = int(gross_us)
        
    except IndexError:
        gross_us = None
    return gross_us

def get_directors_name(soup):
    directors_name = {}
    reg = r'^/people/chart/\?view=Director&id=.+.htm'
    
    try:
        all_a = soup.find_all('a', href= True)

        for a in all_a:
            if re.findall(reg,a['href']):
                directors_name[a.get_text()] = a['href']
            
    except IndexError:
        directors_name = None
    return directors_name

def get_actors_name(soup):
    actors_name = {}
    reg = r'^/people/chart/\?view=Actor&id=.+.htm'
    try:
        all_a = soup.find_all('a', href= True)

        for a in all_a:
            if re.findall(reg,a['href']):
            
                temp = re.sub(r'[*]','',a.get_text())
                actors_name[temp] = a['href']
            
    except IndexError:
        actors_name = None
    return actors_name


def get_director_movies_name(soup,premier):
    director_movies_name = {}
    find_table_reg = r'^/people/chart/\?view=Director&id=.+.htm&sort=title&order=ASC&p=.htm'
    movie_name_reg  = r'^/movies/\?id=.+.htm'
    date_reg = r'\d{1,2}/\d{1,2}/\d{1,2}'
    date_list = []
    movie_link_list = []
    movie_name_list = []
    
    try:
        
        find_table = soup.find_all('a', href = True)
        for table in find_table:
            if re.findall(find_table_reg,table['href']):
                tag_a = table

        tbody = tag_a.find_parent('tbody')
        all_a = tbody.find_all('a', href= True)

        for a in all_a:
            if re.findall(movie_name_reg ,a['href'.encode('utf-8')]):
                movie_link_list.append(a['href'].encode('utf-8'))
                movie_name_list.append(a.get_text().encode('utf-8'))
               
        all_td = tbody.find_all('td')
        for td in all_td:

            if re.findall(date_reg, td.get_text().encode('utf-8')):
                date_list.append(td.get_text().encode('utf-8'))        
                
        for i in range(len(date_list)):
            d = re.split(r'[\/]',date_list[i])
            if int(d[2]) > 30:
                d[2] = '19' + d[2]
            else: 
                d[2] = '20' + d[2]
            date = dt.date(int(d[2]),int(d[0]),int(d[1]))
            
            if date < premier:
                director_movies_name[movie_name_list[i]] = [movie_link_list[i],date]
                
    except Exception as err:
        print(err)
    return director_movies_name

def get_actor_movies_name(soup,premier):
    actor_movies_name = {}
    find_table_reg = r'^/people/chart/\?view=Actor&id=.+.htm&sort=title&order=ASC&p=.htm'
    movie_name_reg  = r'^/movies/\?id=.+.htm'
    date_reg = r'\d{1,2}/\d{1,2}/\d{1,2}'
    date_list = []
    movie_link_list = []
    movie_name_list = []
    
    try:
        
        find_table = soup.find_all('a', href = True)
        for table in find_table:
            if re.findall(find_table_reg,table['href']):
                tag_a = table
        
        tbody = tag_a.find_parent('tbody')
        all_a = tbody.find_all('a', href= True)

        for a in all_a:
            if re.findall(movie_name_reg ,a['href']):
                movie_link_list.append(a['href'])
                movie_name_list.append(a.get_text())
                
        all_td = tbody.find_all('td')
        
        for td in all_td:

            if re.findall(date_reg, td.get_text().encode('utf-8')):
                date_list.append(td.get_text().encode('utf-8'))
            #print date_list
            
        for i in range(len(date_list)):
            d = re.split(r'[\/]',date_list[i])
            if int(d[2]) > 30:
                d[2] = '19' + d[2]
            else: 
                d[2] = '20' + d[2]
            date = dt.date(int(d[2]),int(d[0]),int(d[1]))
            
            if date < premier:  #개봉일보다 이전이면
                actor_movies_name[movie_name_list[i]]  = [movie_link_list[i], date]
                #print actor_movies_name
                
    except Exception as err:
        print(err)
    return actor_movies_name

def get_franchise_link(soup):
    
    reg = r'/franchises/chart/\?id=.+.htm'
    link_list = []
    
    all_a = soup.find_all('a', href= True)
        
    for a in all_a:
        if re.findall(reg,a['href']):

            if re.findall(r'Series',a.get_text()):
                link_list.append(a['href'])

    if link_list:
        franchise_link = link_list[0]
    else:
        franchise_link = None

    return franchise_link

def get_franchise_movies_name(soup):
    
    franchise_movies_name = {}
    find_table_reg = r'^/franchises/chart/\?view=main&id=.+.htm&sort=title&order=ASC&p=.htm'
    movie_name_reg  = r'^/movies/\?id=[\w]+.htm'
    date_reg = r'^/schedule/\?view=bydate&release=theatrical&date=.+&p=.htm'

    find_table = soup.find_all('a', href = True)
    for table in find_table:
        if re.findall(find_table_reg,table['href']):
            tag_a = table

    tbody = tag_a.find_parent('tbody')
    all_a = tbody.find_all('a', href= True)

    for a in all_a:

        if re.findall(movie_name_reg ,a['href']):
            franchise_movies_name[a.get_text()] = a['href']

    return franchise_movies_name

# 영화제목(영어), index 추출해서 dict 만들기

In [12]:

movie_data = load_movie_names("movie.json")

#영화 이름(영어) 추출, mojo링크 추출
movie_name_dict = {}
mojo_link = []
for i in range(699,932):
    
    movie_name_eng= movie_data[i]['Mname']
    movie_name_eng = re.split(r'[(]',movie_name_eng)[0] 
    movie_name_eng = movie_name_eng.encode('utf-8')
    movie_name_dict[i+1] = movie_name_eng
    
    mojo_link.append(movie_data[i]['Webpage'])


# 영화 이름(영어) db에 저장

In [155]:

for i in range(len(movie_name_dict)):
    
    m1 = Movie_Inf()
    m1.set_m_name_e(movie_name_dict[700+i])
    m1.set_movie_id(700+i)
    insert_into_db(m1)

print 'done'

IntegrityError: (_mysql_exceptions.IntegrityError) (1062, "Duplicate entry '700' for key 'PRIMARY'") [SQL: u'INSERT INTO movie_inf (movie_id, m_name_k, grade, is_original, viewing_time, premier, nation, m_name_e, project_budget, gross_kor, gross_us) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'] [parameters: (700, None, None, None, None, None, None, 'Serena', None, None, None)]

# db에서 m_name_e 값가져와서 naver api query로 넘김

In [90]:
rows = select_from_db(Movie_Inf,'')
print 'done'

done


# 개봉일, 상영시간, 국가, 장르db에 저장

In [ ]:
for row in rows:
    url = get_naver_links(row.m_name_e.encode('utf-8'))
    m_name_k = get_korean_title(row.m_name_e.encode('utf-8'))
    span = crawling_naver_movie(url)
    movie_id = row.movie_id
    
    genre_list = get_naver_genre(span)
    for i in range(len(genre_list)):
        g1 = Genre(movie_id, genre_list[i])
        insert_into_db(g1)
    
    viewing_time = get_naver_viewing_time(span)
    premier = get_naver_premier(span)
    nation = get_naver_nation(span)

    values_dict = {}
    values_dict['viewing_time'] = viewing_time
    values_dict['premier'] = premier
    values_dict['nation'] = nation
    values_dict['m_name_k'] = m_name_k
    
    update_into_db(Movie_Inf, Movie_Inf.movie_id == movie_id, values_dict)
    
    

print 'done'

# 원작여부, 등급 저장

In [68]:
for row in rows:
    url = get_naver_links(row.m_name_e.encode('utf-8'))
    movie_id = row.movie_id
    
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html.parser')
    
    grade = get_movie_grade(soup)
    is_original = get_movie_origin(soup)

    values_dict = {}
    if grade:
        values_dict['grade'] = grade.encode('utf-8')
    values_dict['is_original'] = is_original
    
    update_into_db(Movie_Inf, Movie_Inf.movie_id == movie_id, values_dict)
    
    

print 'done'

done


# 영화 link 잘못 찾은것 다시 저장하기

In [105]:
url_list = [(705,  'http://movie.naver.com/movie/bi/mi/basic.nhn?code=139438'),
           (706, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=127359'),
           (709, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=121048'),
           (712, ' http://movie.naver.com/movie/bi/mi/basic.nhn?code=115977'),
           (717, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=107352'),
           (724, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=135705'),
           (732, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=123269'),
           (735, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=123277'),
           (742, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=134830'),
           (744, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=131579'),
           (745, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=94117'),
           (746, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=134982'),
           (747, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=130939'),
           (763, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=115642'),
           (764, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=75426'),
           (770, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=144938'),
           (775, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=136611'),
           (789, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=129461'),
           (794, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=136869'),
           (823, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=134841'),
           (826, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=137341'),
           (836, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=147834'),
           (838,  'http://movie.naver.com/movie/bi/mi/basic.nhn?code=125436'),
           (851, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=137972'),
           (852, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=125419'),
           (857, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=137921'),
           (859, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=134874'),
           (869, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=142369'),
           (875, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=120581'),
           (885, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=145162'), 
           (900, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=151106'),
           (903, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=142333'),
           (912, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=146488'),
           (925, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=139436'), 
           (928, 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=142306')]

In [104]:
for i in range(len(url_list)):
    url = url_list[i][1]
    movie_id = url_list[i][0]
    
    span = crawling_naver_movie(url)
    
    genre_list = get_naver_genre(span)
    for i in range(len(genre_list)):
        g1 = Genre(movie_id, genre_list[i])
        insert_into_db(g1)
    
    viewing_time = get_naver_viewing_time(span)
    premier = get_naver_premier(span)
    nation = get_naver_nation(span)
    
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html.parser')
    grade = get_movie_grade(soup)
    is_original = get_movie_origin(soup)
    m_name_k = get_k_movie_title(soup)

    values_dict = {}
    if grade:
        values_dict['grade'] = grade.encode('utf-8')
    values_dict['is_original'] = is_original
    values_dict['viewing_time'] = viewing_time
    values_dict['premier'] = premier
    values_dict['nation'] = nation
    values_dict['m_name_k'] = m_name_k.encode('utf-8')
    
    update_into_db(Movie_Inf, Movie_Inf.movie_id == movie_id, values_dict)
    
    

print 'done'

done


In [598]:
for i in range(len(mojo_link)):
    
    print mojo_link[i]
    soup = crawling_mojo_movie(mojo_link[i])
    print get_project_budget(soup)
    print get_gross_us(soup)
    korea_gross_link = re.sub(r'\?','?page=intl&', mojo_link[i])
    soup = crawling_mojo_movie(korea_gross_link)
    print get_gross_kor(soup)
    print

/movies/?id=serena.htm
None
176391
24040

/movies/?id=45years.htm
None
4247285
146270

/movies/?id=anomalisa.htm
8000000
3759286
81861

/movies/?id=bythesea.htm
10000000
538460
36704

/movies/?id=sleepingwithotherpeople.htm
None
819431
44671

/movies/?id=youth.htm
None
2703296
538017

/movies/?id=truth.htm
None
2541854
129233

/movies/?id=mistressamerica.htm
None
2500431
52506

/movies/?id=riotclub.htm
None
7734
21819

/movies/?id=assassination15.htm
None
1904682
86915500

/movies/?id=sonofsaul.htm
None
1777043
154070

/movies/?id=99homes.htm
8000000
1411927
92669

/movies/?id=veteran2015.htm
None
1200627
92838964

/movies/?id=5flightsup.htm
None
1020921
213263

/movies/?id=borutonarutothemovie.htm
None
919651
406478

/movies/?id=mustang.htm
None
845464
41974

/movies/?id=seymour.htm
None
743541
138712

/movies/?id=theassassin.htm
None
632542
91089

/movies/?id=whenmarniewasthere.htm
None
561085
228617

/movies/?id=lastnaruto.htm
None
524451
329622

/movies/?id=peggyguggenheimartaddict

# 예산, 미국매출액, 한국매출액 가져와서 db에 저장

In [341]:
for i in range(len(mojo_link)):
    
    movie_id = 700+i
    soup = crawling_mojo_movie(mojo_link[i])
    project_budget = get_project_budget(soup)
    gross_us = get_gross_us(soup)
    
    korea_gross_link = re.sub(r'\?','?page=intl&', mojo_link[i])
    soup = crawling_mojo_movie(korea_gross_link)
    gross_kor = get_gross_kor(soup)
    
    values_dict = {}
    if project_budget:
        values_dict['project_budget'] = project_budget
    values_dict['gross_us'] = gross_us
    if gross_kor:
        values_dict['gross_kor'] = gross_kor
    
    update_into_db(Movie_Inf, Movie_Inf.movie_id == movie_id, values_dict)

# 감독, 감독영화, 감독 영화 매출액 가져와서 db에 저장

In [348]:
for i in range(len(mojo_link)):
    soup = crawling_mojo_movie(mojo_link[i])
    directors_name = get_directors_name(soup)
    print directors_name


{}
{}
{u'Charlie Kaufman': u'/people/chart/?view=Director&id=charliekaufman.htm'}
{u'Angelina Jolie': u'/people/chart/?view=Director&id=angelinajolie.htm'}
{u'Leslye Headland': u'/people/chart/?view=Director&id=leslyeheadland.htm'}
{}
{u'James Vanderbilt': u'/people/chart/?view=Director&id=jamesvanderbilt.htm'}
{u'Noah Baumbach': u'/people/chart/?view=Director&id=noahbaumbach.htm'}
{u'Lone Scherfig': u'/people/chart/?view=Director&id=lonescherfig.htm'}
{}
{}
{}
{}
{}
{}
{}
{u'Ethan Hawke': u'/people/chart/?view=Director&id=ethanhawke.htm'}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{u'David Gordon Green': u'/people/chart/?view=Director&id=davidgordongreen.htm'}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{u'Eli Roth': u'/people/chart/?view=Director&id=eliroth.htm'}
{}
{}
{}
{}
{}
{}
{}
{}
{u'Anthony Russo': u'/people/chart/?view=Director&id=anthonyrusso.htm', u'Joe Russo': u'/people/chart/?view=Director&id=joerusso.htm'}
{u'Gareth Edwards': u'/people/chart/?view=Director&id=garethedwar

In [40]:
dir_num = 0

for i in range(len(mojo_link)):
    print
    movie_id = 700+i
    print movie_id
    
    directors_name = []
    directors_link = []

    soup = crawling_mojo_movie(mojo_link[i])
    directors= get_directors_name(soup)
    directors_name = directors.keys()
    directors_link = directors.values()
    
    if directors_name:
        result = select_from_db(Movie_Inf.premier, Movie_Inf.movie_id == movie_id)
        if result != [(None,)]:
            for row in result:
                row = re.sub(r'[(datetime.date()\s]','',str(row))
                dt1 = dt.datetime.strptime(row,'%Y,%m,%d,')
                premier = dt.date(dt1.year, dt1.month, dt1.day)
                #print premier
        else:
            premier = dt.date.today()
            
        for j in range(len(directors_link)):
            dir_num += 1
            dr = Director()
            dr.set_director_id(dir_num)
            dr.set_movie_id(700+i)
            dr.set_director(directors_name[j])
            insert_into_db(dr)
            
            soup = crawling_mojo_movie(directors_link[j])
            director_movies = get_director_movies_name(soup, premier)

            director_movies_name = director_movies.keys()
            director_movies_info = director_movies.values()
        
            if director_movies_name:
                for k in range(len(director_movies_name)):

                    korea_gross_link = re.sub(r'\?','?page=intl&', director_movies_info[k][0].encode('utf-8'))
                    soup = crawling_mojo_movie(korea_gross_link)
                    gross = get_gross_kor(soup)
                    
                    dr_m = Director_Movies()
                    if gross:
                        dr_m.set_director_id(dir_num)
                        dr_m.set_director_movie(director_movies_name[k])
                        dr_m.set_gross(gross)
                        dr_m.set_premier(director_movies_info[k][1])
                        print dir_num
                        print director_movies_info[k][0]
                        print director_movies_name[k][1]
                        print gross
                        
                        insert_into_db(dr_m)
    
    



700

701

702
1
/movies/?id=synecdochenewyork.htm
y
27681
1
/movies/?id=anomalisa.htm
n
81861

703
2
/movies/?id=bythesea.htm
y
36704
2
/movies/?id=unbroken.htm
n
1478961

704
3
/movies/?id=sleepingwithotherpeople.htm
l
44671

705

706
4
/movies/?id=truth.htm
r
129233

707
5
/movies/?id=whilewereyoung.htm
h
349945
5
/movies/?id=francisha.htm
r
431554
5
/movies/?id=mistressamerica.htm
i
52506

708
6
/movies/?id=aneducation.htm
n
14062
6
/movies/?id=oneday.htm
n
698542
6
/movies/?id=riotclub.htm
h
21819

709

710

711

712

713

714

715

716
7
/movies/?id=seymour.htm
e
138712
7
/movies/?id=hotteststate.htm
h
72384

717

718

719

720

721

722

723

724

725

726

727

728

729

730

731

732

733
8
/movies/?id=manglehorn.htm
a
10440

734

735

736

737

738

739

740

741

742

743

744

745

746

747
9
/movies/?id=hostel2.htm
o
5274
9
/movies/?id=knockknock.htm
n
22061

748

749

750

751

752

753

754

755

756
10
/movies/?id=welcometocollinwood.htm
e
409213
10
/movies/?id=marvel14

# 배우, 배우영화, 배우 영화 매출액 가져와서 db에 저장

In [ ]:
act_num = 0

for i in range(len(mojo_link)):
    print
    movie_id = 700+i
    print movie_id
    
    actors_name = []
    actors_link = []

    soup = crawling_mojo_movie(mojo_link[i])
    actors = get_actors_name(soup)
    actors_name = actors.keys()
    actors_link = actors.values()
    print actors_name
    
    if actors_name:

        result = select_from_db(Movie_Inf.premier, Movie_Inf.movie_id == movie_id)
        if result != [(None,)]:
            for row in result:
                row = re.sub(r'[(datetime.date()\s]','',str(row))
                dt1 = dt.datetime.strptime(row,'%Y,%m,%d,')
                premier = dt.date(dt1.year, dt1.month, dt1.day)
                print premier
        else:
            premier = dt.date.today()
            
        for j in range(len(actors_link)):
            act_num += 1
            at = Actor()
            at.set_actor_id(act_num)
            at.set_movie_id(700+i)
            at.set_actor(actors_name[j])
            insert_into_db(at)
            
            soup = crawling_mojo_movie(actors_link[j])
            print actors_link[j]
            print
            actor_movies= get_actor_movies_name(soup, premier)

            actor_movies_name = actor_movies.keys()
            actor_movies_info = actor_movies.values()
            
            if actor_movies_name:
                for k in range(len(actor_movies_name)):
                    korea_gross_link = re.sub(r'\?','?page=intl&', actor_movies_info[k][0].encode('utf-8'))
                    soup = crawling_mojo_movie(korea_gross_link)
                    gross = get_gross_kor(soup)
                    at_m = Actor_Movies()
                    if gross:
                        at_m.set_actor_id(act_num)
                        at_m.set_actor_movie(actor_movies_name[k])
                        at_m.set_gross(gross)
                        at_m.set_premier(actor_movies_info[k][1])
                        print act_num
                        print actor_movies_info[k][0]
                        print gross
                        print actor_movies_info[k][1]
                        
                        insert_into_db(at_m)
    
    


In [101]:

for i in range(175, len(mojo_link)):
    print
    movie_id = 700+i
    print movie_id
    
    actors_name = []
    actors_link = []

    soup = crawling_mojo_movie(mojo_link[i])
    actors = get_actors_name(soup)
    actors_name = actors.keys()
    actors_link = actors.values()
    print actors_name
    
    if actors_name:

        result = select_from_db(Movie_Inf.premier, Movie_Inf.movie_id == movie_id)
        if result != [(None,)]:
            for row in result:
                row = re.sub(r'[(datetime.date()\s]','',str(row))
                dt1 = dt.datetime.strptime(row,'%Y,%m,%d,')
                premier = dt.date(dt1.year, dt1.month, dt1.day)
                print premier
        else:
            premier = dt.date.today()
            
        for j in range(len(actors_link)):
            act_num += 1
            at = Actor()
            at.set_actor_id(act_num)
            at.set_movie_id(700+i)
            at.set_actor(actors_name[j])
            insert_into_db(at)
            
            soup = crawling_mojo_movie(actors_link[j])
            print actors_link[j]
            print
            actor_movies= get_actor_movies_name(soup, premier)

            actor_movies_name = actor_movies.keys()
            actor_movies_info = actor_movies.values()
            
            if actor_movies_name:
                for k in range(len(actor_movies_name)):
                    korea_gross_link = re.sub(r'\?','?page=intl&', actor_movies_info[k][0].encode('utf-8'))
                    soup = crawling_mojo_movie(korea_gross_link)
                    gross = get_gross_kor(soup)
                    at_m = Actor_Movies()
                    if gross:
                        at_m.set_actor_id(act_num)
                        at_m.set_actor_movie(actor_movies_name[k])
                        at_m.set_gross(gross)
                        at_m.set_premier(actor_movies_info[k][1])
                        print act_num
                        print actor_movies_info[k][0]
                        print gross
                        print actor_movies_info[k][1]
                        
                        insert_into_db(at_m)
                        print 'inserted'
    
    



875
[u'Gael Garcia Bernal', u'Jeffrey Dean Morgan']
2016-10-05
/people/chart/?view=Actor&id=gaelgarciabernal.htm

454
/movies/?id=scienceofsleep.htm
403950
2006-09-22
inserted
454
/movies/?id=king.htm
17100
2006-05-19
inserted
454
/movies/?id=limitsofcontrol.htm
20288
2009-05-01
inserted
454
/movies/?id=badeducation.htm
138752
2004-11-19
inserted
454
/movies/?id=babel.htm
1447031
2006-10-27
inserted
454
/movies/?id=blindness08.htm
3312141
2008-10-03
inserted
454
/movies/?id=motorcyclediaries.htm
207899
2004-09-24
inserted
/people/chart/?view=Actor&id=jeffreydeanmorgan.htm

455
/movies/?id=heist2015.htm
80262
2015-11-13
inserted
455
/movies/?id=watchmen.htm
3053729
2009-03-06
inserted
455
/movies/?id=psiloveyou.htm
2265760
2007-12-21
inserted
455
/movies/?id=shanghai.htm
1763090
2015-10-02
inserted
455
/movies/?id=takingwoodstock.htm
85101
2009-08-26
inserted

876
[u'Don Cheadle', u'Ewan McGregor']
2016-08-10
/people/chart/?view=Actor&id=doncheadle.htm

456
/movies/?id=ironman2.htm
270

In [100]:
act_num =453
print act_num

453


# 시리즈 영화, 시리즈 영화 매출액 가져와서 db에 저장

In [60]:
for i in range(len(mojo_link)):
    print
    movie_id = 700+i
    print movie_id
    
    soup = crawling_mojo_movie(mojo_link[i])
    franchise_link = get_franchise_link(soup)
    print franchise_link
        
    if franchise_link:
        soup = crawling_mojo_movie(franchise_link)
        franchise_movies = get_franchise_movies_name(soup)
        franchise_movies_name = franchise_movies.keys()
        franchise_movies_link = franchise_movies.values()
    
        for i in range(len(franchise_movies_link)):
  
            korea_gross_link = re.sub(r'\?','?page=intl&', franchise_movies_link[i])
            soup = crawling_mojo_movie(korea_gross_link)
            gross = get_gross_kor(soup)
            
            f = Franchise()
            
            f.set_movie_id(movie_id)
            f.set_franchise_movie(franchise_movies_name[i])
            if gross:
                f.set_gross(gross)
            insert_into_db(f)


700
None

701
None

702
None

703
None

704
None

705
None

706
None

707
None

708
None

709
None

710
None

711
None

712
None

713
None

714
None

715
None

716
None

717
None

718
None

719
None

720
None

721
None

722
None

723
None

724
None

725
None

726
None

727
None

728
None

729
None

730
None

731
None

732
None

733
None

734
None

735
None

736
None

737
None

738
None

739
None

740
None

741
None

742
None

743
None

744
None

745
None

746
None

747
None

748
None

749
None

750
None

751
None

752
None

753
None

754
None

755
None

756
/franchises/chart/?id=captainamerica.htm

757
/franchises/chart/?id=starwars.htm

758
None

759
None

760
None

761
/franchises/chart/?id=petsfranch.htm

762
/franchises/chart/?id=batman.htm

763
/franchises/chart/?id=fantasticbeasts.htm

764
/franchises/chart/?id=xmen.htm

765
/franchises/chart/?id=dc.htm

766
/franchises/chart/?id=avengers.htm

767
None

768
/franchises/chart/?id=xmen.htm

769
/franchises/chart/?id=kungfupanda.ht